In [1]:
# ingest.py
import os
import json
import chromadb
from sentence_transformers import SentenceTransformer

# langchain 'text_splitter' import compatibility — some langchain v1+ releases have changed
# the package layout and don't include the old `langchain.text_splitter` module. To be
# resilient, try to import it and fall back to a lightweight splitter implementation.
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
except Exception:
    # Minimal fallback TextSplitter compatible API (split_text method)
    class RecursiveCharacterTextSplitter:
        def __init__(self, chunk_size=1000, chunk_overlap=100):
            self.chunk_size = chunk_size
            self.chunk_overlap = chunk_overlap
        def split_text(self, text):
            if not isinstance(text, str):
                text = str(text)
            out = []
            start = 0
            while start < len(text):
                end = start + self.chunk_size
                out.append(text[start:end])
                start += max(1, self.chunk_size - self.chunk_overlap)
            return out

c:\Users\koush\PyProjects\ChatBot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_json_knowledge_base(filepath="knowledge_base.json"):
    """
    Loads pre-chunked documents with metadata from a single JSON file.
    The JSON file should be a list of objects, where each object has:
    - "chunk_id": A unique string identifier.
    - "chunk_text": The text content of the chunk.
    - "metadata": A dictionary of metadata.
    """
    print(f"Loading knowledge from JSON file: '{filepath}'...")
    if not os.path.exists(filepath):
        print(f"Error: The file '{filepath}' was not found. Please create it.")
        return []

    try:
        # Ensure the file is read with UTF-8 encoding for Farsi characters
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"Successfully loaded {len(data)} knowledge chunks.")
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {filepath}. Please check the file format. Details: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred while reading {filepath}: {e}")
        return []

In [3]:
def run():
    """
    Main function to ingest documents from JSON and store them in ChromaDB.
    """
    print("--- Starting Knowledge Base Ingestion from JSON ---")

    # 1. Load documents from JSON. This data is already chunked.
    documents_data = load_json_knowledge_base()
    if not documents_data:
        print("No documents to ingest. Exiting.")
        return

    # Extract the necessary components for ChromaDB
    original_texts = [doc["chunk_text"] for doc in documents_data]
    document_ids = [doc["chunk_id"] for doc in documents_data]
    metadatas = [doc["metadata"] for doc in documents_data]

    # ENRICH FOR EMBEDDING
    enriched_texts = []
    for doc in documents_data:
        meta = doc["metadata"]
        # Build natural Farsi context
        context = f"[بخش: {meta.get('source_title')}]"
        if meta.get("parent_category"):
            context += f" [زیرمجموعه: {meta['parent_category']}]"
        enriched = f"{context}\n{doc['chunk_text']}"
        enriched_texts.append(enriched)



    # --- VERIFICATION STEP 1: Inspect the loaded data ---
    print("\n--- Verification: Inspecting first loaded chunk ---")
    if documents_data:
        print(f"Chunk ID: {document_ids[0]}")
        print(f"Chunk Text: \"{original_texts[0][:150]}...\"")
        print(f"Chunk Metadata: {metadatas[0]}")
    print("------------------------------------------------\n")

    # 2. Initialize the embedding model
    print("Initializing embedding model (this may take a moment)...")
    model_name = "paraphrase-multilingual-mpnet-base-v2"
    embedding_model = SentenceTransformer(model_name)
    print("Embedding model initialized.")

    # 3. Setup ChromaDB client
    client = chromadb.PersistentClient(path="./chroma_db")

    # 4. Create or get the collection
    collection_name = "farsi_rag_collection"
    print(f"Setting up ChromaDB collection: '{collection_name}'...")
    # Delete the collection if it already exists to ensure a fresh start
    if collection_name in [c.name for c in client.list_collections()]:
        client.delete_collection(name=collection_name)
        print(f"Deleted existing collection '{collection_name}' for a fresh start.")
    collection = client.get_or_create_collection(name=collection_name)
    print("Collection setup complete.")

    # 5. Generate embeddings for all document chunks
    print("Generating embeddings for all document chunks...")
    embeddings = embedding_model.encode(enriched_texts, show_progress_bar=True)

    # --- VERIFICATION STEP 2: Inspect the embeddings ---
    print("\n--- Verification: Inspecting embeddings ---")
    print(f"Shape of embeddings array: {embeddings.shape}")
    print(f"Sample of first embedding vector: {embeddings[0][:5]}...")
    print("------------------------------------------\n")

    # 6. Add to ChromaDB collection (NOW WITH METADATA!)
    print("Adding documents with their metadata to ChromaDB...")
    collection.add(
        embeddings=embeddings.tolist(),
        documents=original_texts,
        metadatas=metadatas,  # <-- Here we add the rich metadata
        ids=document_ids
    )

    # --- VERIFICATION STEP 3: Test a query ---
    print("\n--- Verification: Performing a test query ---")
    test_query = "خدمات شرکت چیست؟"
    print(f"Querying with text: \"{test_query}\"")

    test_query_embedding = embedding_model.encode(test_query).tolist()

    results = collection.query(
        query_embeddings=[test_query_embedding],
        n_results=1
    )

    retrieved_doc = results['documents'][0][0]
    retrieved_meta = results['metadatas'][0][0]
    print(f"Retrieved document: \"{retrieved_doc[:150]}...\"")
    print(f"Retrieved metadata: {retrieved_meta}")
    print("--------------------------------------------\n")

    print("\n--- Ingestion Complete ---")
    print(f"Successfully added {len(documents_data)} documents to the '{collection_name}' collection.")
    print("You can now run 'app.py' to start the chatbot.")

In [4]:
run()

--- Starting Knowledge Base Ingestion from JSON ---
Loading knowledge from JSON file: 'knowledge_base.json'...
Successfully loaded 99 knowledge chunks.

--- Verification: Inspecting first loaded chunk ---
Chunk ID: contact_us_001
Chunk Text: "راه‌های تماس با ما به شرح زیر است:
- تلفن همراه: ۰۹۹-۸۱۸۱-۸۹۷۸
- تلفن ثابت: ۰۱۱-۴۲۴۳-۳۴۴۳
- فکس: ۰۱۱-۴۲۴۳-۳۴۴۴
- ایمیل: info@tabenergy.ir
- آدرس: مازن..."
Chunk Metadata: {'source_title': 'تماس با ما', 'source_url': 'tabenergy.ir/contact/'}
------------------------------------------------

Initializing embedding model (this may take a moment)...
Embedding model initialized.
Setting up ChromaDB collection: 'farsi_rag_collection'...
Collection setup complete.
Generating embeddings for all document chunks...


Batches: 100%|██████████| 4/4 [00:11<00:00,  2.77s/it]



--- Verification: Inspecting embeddings ---
Shape of embeddings array: (99, 768)
Sample of first embedding vector: [-0.03597982 -0.03558146 -0.01376151 -0.04435126  0.12509173]...
------------------------------------------

Adding documents with their metadata to ChromaDB...

--- Verification: Performing a test query ---
Querying with text: "خدمات شرکت چیست؟"
Retrieved document: "سوال: آدرس شرکت کجاست؟
پاسخ: آدرس ما: مازندران، سوادکوه شمالی، شهرک صنعتی بشل، فاز ۲، همت ۷...."
Retrieved metadata: {'source_title': 'آدرس شرکت کجاست؟', 'source_url': 'tabenergy.ir/contact/', 'parent_category': 'پرسش و پاسخ های رایج'}
--------------------------------------------


--- Ingestion Complete ---
Successfully added 99 documents to the 'farsi_rag_collection' collection.
You can now run 'app.py' to start the chatbot.
